In [8]:
#initialize parametrized variables

experimentID = ''
runID = ''
n = 0

In [9]:
import pandas as pd
import numpy as np

import mlflow
from mlflow.pyfunc import PythonModel

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

In [10]:
#sets the MLflow experiment and active run
mlflow.set_experiment(experimentID)
mlflow.start_run(runID)

<ActiveRun: >

In [11]:
%store -r data
%store -r model

In [20]:
data

,Input,Intent,sentence_vector
779,Ich möchte gerne Genaueres zum Thema smart wel...,DE_apps_smart_world,"[-0.8428768, -0.37270302, -0.37412453, 0.34199..."
2143,smart EQ kaufen,DE_buying_smart,"[-0.5210464, -0.80692446, 0.7015624, 0.3663871..."
45,Kann ich @accessories:rims kaufen?,DE_accessories_rims_tires,"[-0.4768655, -0.80895996, -0.22446331, 0.23924..."
823,Wo kann ich die smart apps herunterladen?,DE_apps_unspecified,"[-1.0971125, -0.28539774, 0.41622654, 0.389162..."
2676,Wie bezahle ich die Ladung an Ladestationen?,DE_charging_plugsurfing_pay,"[-0.61764973, -0.06500912, -0.10014312, 0.0754..."
...,...,...,...
1665,smart Werksführung,DE_brand_factorytour,"[-1.3307184, -0.28871033, 0.089040525, 0.64579..."
3205,Bist du gutaussehend,DE_chitchat_botappearance,"[-0.2327941, -0.15047005, 0.023479793, 0.56771..."
617,Ich möchte gerne smart media connect Dienste i...,DE_apps_live_traffic,"[-0.46775913, -0.73983145, -0.35762158, -0.038..."
1851,Welche Slogans stehen für smart,DE_brand_slogan,"[-0.30988413, -0.43118307, 0.35115284, 0.15418..."


In [16]:
in_df = pd.DataFrame(list(data[vector_series_colname]), index=data.index)
in_df

,0,1,2,3,4,5,6,7,8,9,...,758,759,760,761,762,763,764,765,766,767
779,-0.842877,-0.372703,-0.374125,0.341997,0.243520,0.326461,-0.954818,0.303712,-0.243708,-0.290412,...,0.487764,-0.533116,-1.258471,-0.779488,0.654580,-0.510307,-0.035221,1.010295,1.116711,0.075714
2143,-0.521046,-0.806924,0.701562,0.366387,0.422554,-0.085528,-1.440300,0.510826,0.137715,-0.253847,...,-0.622909,0.649267,-0.976162,-0.390562,0.694900,-0.230558,0.157001,0.868921,0.301646,-0.323377
45,-0.476866,-0.808960,-0.224463,0.239249,-0.519830,0.113507,-0.563005,0.659767,-0.168730,-0.547951,...,0.232365,-0.617275,-1.158470,-0.588344,0.963783,0.328754,-0.242162,1.339786,0.845982,-0.158999
823,-1.097113,-0.285398,0.416227,0.389162,-1.090829,0.429182,-0.517741,0.449940,-0.053429,-0.190996,...,0.264644,-0.321717,-1.239357,-0.737600,0.467495,-0.452072,-0.443670,1.500103,1.227222,-0.233938
2676,-0.617650,-0.065009,-0.100143,0.075442,-0.251117,-0.094384,-0.421853,0.466900,-0.061235,-0.337533,...,0.007621,-0.021059,-0.885014,-0.780684,0.389176,-0.050969,-0.516177,1.575726,0.853910,0.239138
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1665,-1.330718,-0.288710,0.089041,0.645795,0.569244,-0.105619,-1.176157,0.131829,0.619492,-0.451358,...,0.234125,0.276488,-0.580507,-0.256872,0.347538,0.238654,0.008228,0.548552,0.853676,-0.087213
3205,-0.232794,-0.150470,0.023480,0.567720,0.101463,1.079851,-0.904403,0.144455,0.542775,0.130609,...,0.380965,-0.124144,-0.221780,-0.240669,0.070306,0.104509,0.235491,1.026465,1.139424,-0.348386
617,-0.467759,-0.739831,-0.357622,-0.038354,-0.047169,0.538571,-1.308450,0.504528,-0.541302,-0.704315,...,0.104981,-0.657608,-1.011291,-0.993735,0.353765,-0.412378,-0.659039,1.278163,0.538435,-0.001492
1851,-0.309884,-0.431183,0.351153,0.154184,-0.312764,-0.215552,-0.866382,0.333930,0.299800,-0.111766,...,-0.000389,0.126486,-0.954762,-0.210149,0.501020,-0.937833,-0.006785,0.676590,0.484697,0.160951


In [17]:
class STMPredProba(PythonModel):
    
    def __init__(self, project_id, runID, model, n):
        self._project_id = project_id
        self._runID = runID
        self._model = model
        self._n = n

    def predict(self, X, n=0):#, pipe_model, n=0):
        prob_df = pd.DataFrame(self._model.predict_proba(X), index=X.index, columns=self._model.classes_)
        if n ==0:
            
            return prob_df
        else:
            top_n = np.argpartition(prob_df.transpose(), -self._n, axis=0)[-self._n:].apply(lambda x:
                                                                                  prob_df.transpose().index.values[
                                                                                      x]).transpose()
            top_n.columns = ["{}".format(i+1) for i in range(self._n)]
            prob_matrix = pd.DataFrame()
            for col in top_n.columns:
                prob_matrix[col] = prob_df.apply(lambda row: row[top_n[col][row.name]], axis=1)
            top_prob = pd.merge(top_n, prob_matrix, right_index=True, left_index=True)
            tuple_class_prob = pd.DataFrame()
            for i in range(1, self._n+1): #constructs the tuple table
                runner_x = '_'.join([str(i), 'x'])
                runner_y = '_'.join([str(i), 'y'])
                tuple_class_prob[i] = pd.DataFrame([top_prob[runner_x], top_prob[runner_y]]).transpose().apply(tuple,
                                                                                                           axis=1)
            tuple_class_prob.columns = [f"top{i+1}"+str(i) for i in range(1, self._n+1)]
            for i in range(len(tuple_class_prob)): #sorts the tuple table
                list_to_sort=tuple_class_prob.iloc[i].tolist()
                list_to_sort.sort(key=lambda x: -x[1])
                tuple_class_prob.iloc[i]=list_to_sort
            return tuple_class_prob


In [21]:
pred=STMPredProba(project_id=project_id, runID=runID, model=model,n=n)

In [22]:
pred.predict(X=in_df)

,DE_accessories_rims_tires,DE_android_car_general_info
779,0.691543,0.308457
2143,0.901415,0.098585
45,0.994926,0.005074
823,0.546550,0.453450
2676,0.975508,0.024492
...,...,...
1665,0.849980,0.150020
3205,0.951173,0.048827
617,0.470685,0.529315
1851,0.799135,0.200865


In [ ]:
mlflow.end_run()